In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
from sqlalchemy import create_engine

# CONFIGURATION

In [ ]:
CSV_FILE = "../data/sales_2.csv"

In [ ]:
MYSQL_USER = "<user_name>"
MYSQL_PASSWORD = "<password>"
MYSQL_HOST = "localhost"
MYSQL_PORT = "3306"
MYSQL_DATABASE = "sales_db"

TARGET_TABLE = "etl_transformed_sales"

## Database Connection

In [ ]:
# SQLAlchemy connection string
conn_str = (
    f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DATABASE}"
)

In [ ]:
# Create SQLAlchemy engine
engine = create_engine(conn_str)

### STEP 1: READ CSV FILE (Extract)

In [ ]:
try:
    df = pd.read_csv(CSV_FILE)
    print(f"Data extracted from {CSV_FILE}, shape = {df.shape}")
    display(df.head())
except Exception as e:
    print(f"Error reading CSV: {e}")
    raise

## STEP 2: TRANSFORMATIONS (Transform)

In [ ]:
# Clean column names
df.columns = (
    df.columns
      .str.replace(' ', '_')
      .str.replace(r'([A-Z])', r'_\1', regex=True)
      .str.lower()
      .str.strip('_')
)

In [ ]:
# Convert order_date → datetime64
df["order_date"] = pd.to_datetime(df["order_date"])

In [ ]:
# numeric conversion
df["sales_amount"] = pd.to_numeric(df["sales_amount"])

In [ ]:
# Derived field
df["unit_price"] = df["sales_amount"]

In [ ]:
# Categorical tier
conditions = [
    df["sales_amount"] >= 500,
    df["sales_amount"] >= 100
]
choices = ["High Value", "Medium Value"]

df["sales_tier"] = np.select(conditions, choices, default="Low Value")

In [ ]:
# Filter out non-positive values
df = df[df["sales_amount"] > 0]

In [ ]:
# Add load timestamp
df["load_timestamp"] = pd.to_datetime(datetime.utcnow())

In [ ]:
print("Transformations complete.")
display(df.head())

## STEP 3: LOAD INTO MYSQL (Load)

In [ ]:
try:
    df.to_sql(
        name=TARGET_TABLE,
        con=engine,
        if_exists="append",
        index=False,
        chunksize=1000
    )

    print(f"Data successfully loaded into MySQL table: {TARGET_TABLE}")

except Exception as e:
    print(f"Error loading to MySQL: {e}")